LangChain Academy: https://academy.langchain.com/


In [1]:
!pip install langchain-openai langchain-community openai google-search-results

In [2]:
!pip install python-dotenv yfinance

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import yfinance as yf
from langchain_core.tools import tool
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Read the key
#api_key = os.getenv("OPENAI_API_KEY")

In [4]:
load_dotenv()

True

Fetching stock price of a certain company (say, SUZLON)

In [5]:

def get_suzlon_price() -> float:
    """
    Gets the price of Suzlon from National Stock Exchange using yfinance function Ticker()

    Returns:
    stock price of Suzlon from National Stock Exchange
    """
    ticker = yf.Ticker("SUZLON.NS")
    info = ticker.info or {}

    price = info.get("regularMarketPrice") or info.get("previousClose")
    if price:
        return float(price)

    hist = ticker.history(period="2d")
    if not hist.empty:
        return float(hist["Close"].iloc[-1])

    raise RuntimeError("Could not fetch stock price")

Compute stock value

In [6]:

def compute_value(shares: float, cost: float) -> tuple:
    """
    Compute Suzlon portfolio metrics with production-grade error handling.

    Args:
        shares: Number of shares held (must be non-negative).
        cost: Average cost per share.
        get_price_func: Function to fetch current Suzlon price (injected for testability).
        logger: Optional logger instance.

    Returns:
        PortfolioMetrics dataclass with computed values.
    """
    price = get_suzlon_price()
    total_value = shares * price
    total_cost = shares * cost
    profit = total_value - total_cost
    roi = (profit / total_cost * 100) if total_cost else 0
    print("My Tool Used not LLM")
    return price, total_value, total_cost, profit, roi


LangChain LLM Pipeline (LCEL)

In [7]:
prompt = ChatPromptTemplate.from_template("""
You are a financial assistant.

A user owns {shares} shares of SUZLON bought at ₹{cost} each.

Current market price: ₹{price}
Total value: ₹{total_value}
Total cost: ₹{total_cost}
Profit/Loss: ₹{profit}
ROI: {roi}%

Write a **clear, friendly explanation** of these results.
""")

Forming LCEL pipeline

In [8]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()
chain = prompt | model | parser

Calculating Loss or gain

In [9]:
shares = 10
cost = 40
price, total_value, total_cost, profit, roi = compute_value(shares, cost)

result = chain.invoke({
        "shares": shares,
        "cost": cost,
        "price": f"{price:.2f}",
        "total_value": f"{total_value:.2f}",
        "total_cost": f"{total_cost:.2f}",
        "profit": f"{profit:.2f}",
        "roi": f"{roi:.2f}"
    })

print(result)

My Tool Used not LLM
Sure! Let's break down your investment in SUZLON and what the numbers mean for you.

1. **Shares Owned**: You own 10 shares of SUZLON, which means you have a stake in the company.

2. **Purchase Price**: You bought each share at ₹40. So, your total investment (or cost) for these 10 shares is:
   - 10 shares x ₹40 = ₹400.

3. **Current Market Price**: The current market price of SUZLON shares is ₹53.62. This is the price at which you could sell your shares today.

4. **Total Value**: If you were to sell all your shares at the current market price, you would receive:
   - 10 shares x ₹53.62 = ₹536.20.

5. **Profit/Loss**: To find out how much you've made (or lost), we subtract your total cost from the total value:
   - Total Value (₹536.20) - Total Cost (₹400.00) = Profit of ₹136.20. 
   This means you have made a profit of ₹136.20 on your investment.

6. **Return on Investment (ROI)**: This is a way to measure how well your investment has performed. It’s calculated 